In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Settings:
pd.set_option('display.width', 190)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('max_colwidth', 200)
pd.options.display.float_format = '{:.4f}'.format
plt.style.use('default')
np.set_printoptions(threshold = 30, edgeitems = 30, precision = 2, suppress = False)

In [3]:
df = pd.read_csv("../original_data/gdp_m.csv")
df = df.rename({"date": "Date", "gdp": "GDP"}, axis=1)
df.Date = pd.to_datetime(df.Date)
# df = df.set_index("Date")
df = df.drop(columns=['year'])
df.head()

,Date,GDP
0,1947-01-01,243.1640
1,1947-04-01,245.9680
2,1947-07-01,249.5850
3,1947-10-01,259.7450
4,1948-01-01,265.7420


In [4]:
bci = pd.read_csv("../original_data/BCI-values.csv")
bci.Date = pd.to_datetime(bci.Date, dayfirst=True)

# log transformation
# bci.BCI = bci.BCI.apply(lambda x: np.log(x))
# bci.BCIg = bci.BCIg.apply(lambda x: np.log(x + abs(min(bci.BCIg)) + 0.1)) # Because there are values that are negative, we transform all values just a little bit positive to be able to make a logarithmic transformation.
# bci.BCIp = bci.BCIp.apply(lambda x: np.log(x + abs(min(bci.BCIp)) + 0.1))

In [5]:
df.Date = df.Date + pd.DateOffset(months=1, days=-1)
df = df[df['Date'] > bci.Date[0] + pd.DateOffset(months=-3)]
df = df.reset_index(drop=True)
df.head()

,Date,GDP
0,1967-01-31,844.1700
1,1967-04-30,848.9830
2,1967-07-31,865.2330
3,1967-10-31,881.4390
4,1968-01-31,909.3870


In [6]:
lengths = []
for i in range(len(df) - 1):
    date1 = df['Date'][i]
    date2 = df['Date'][i + 1]
    length = len(bci[(bci.Date > date1) & (bci.Date <= date2)])
    if length != 0:
        lengths.append(length)
max(lengths)

14

In [7]:
for i in range(14):
    df[f'BCI_{i}'] = np.nan
    df[f'BCIp_{i}'] = np.nan
    df[f'BCIg_{i}'] = np.nan
    
for i in range(len(df) - 1):
    date1 = df['Date'][i]
    date2 = df['Date'][i + 1]
    temp = bci[(bci.Date > date1) & (bci.Date <= date2)]
    for idx in range(len(temp)):
        df = df.copy()
        df[f'BCI_{idx}'].iloc[i] = temp['BCI'].iloc[idx]
        df[f'BCIp_{idx}'].iloc[i] = temp['BCIp'].iloc[idx]
        df[f'BCIg_{idx}'].iloc[i] = temp['BCIg'].iloc[idx]

# TODO: better fillna
df = df.fillna(method="ffill", axis=1)
df.head()

/opt/homebrew/Caskroom/miniconda/base/envs/ds/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,Date,GDP,BCI_0,BCIp_0,BCIg_0,BCI_1,BCIp_1,BCIg_1,BCI_2,BCIp_2,BCIg_2,BCI_3,BCIp_3,BCIg_3,BCI_4,BCIp_4,BCIg_4,BCI_5,BCIp_5,BCIg_5,BCI_6,BCIp_6,BCIg_6,BCI_7,BCIp_7,BCIg_7,BCI_8,BCIp_8,BCIg_8,BCI_9,BCIp_9,BCIg_9,BCI_10,BCIp_10,BCIg_10,BCI_11,BCIp_11,BCIg_11,BCI_12,BCIp_12,BCIg_12,BCI_13,BCIp_13,BCIg_13
0,1967-01-31,844.1700,100.0000,100.0000,0.0000,100.0000,99.5000,0.0000,99.6000,93.1000,0.0000,99.8000,96.4000,0.0000,99.9000,98.7000,0.0000,99.7000,94.6000,0.0000,99.9000,98.4000,0.0000,99.9000,97.6000,0.0000,99.2000,85.8000,0.0000,98.5000,75.1000,0.0000,98.9000,82.2000,0.0000,99.8000,96.6000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,1967-04-30,848.9830,100.4000,100.0000,0.0000,100.5000,100.0000,0.0000,100.1000,93.4000,0.0000,99.8000,88.3000,0.0000,100.2000,95.6000,0.0000,100.6000,100.0000,0.0000,101.4000,100.0000,0.0000,101.8000,100.0000,0.0000,102.0000,100.0000,0.0000,102.3000,100.0000,0.0000,103.1000,100.0000,0.0000,103.2000,100.0000,0.0000,103.2000,99.8000,0.0000,0.0000,0.0000,0.0000
2,1967-07-31,865.2330,103.4000,100.0000,0.0000,103.4000,99.9000,0.0000,103.4000,100.0000,0.0000,103.6000,100.0000,0.0000,103.5000,98.0000,0.0000,104.0000,100.0000,0.0000,104.3000,100.0000,0.0000,104.9000,100.0000,0.0000,104.9000,100.0000,0.0000,104.7000,96.7000,0.0000,104.9000,99.7000,0.0000,104.5000,93.0000,0.0000,104.5000,93.1000,0.0000,0.0000,0.0000,0.0000
3,1967-10-31,881.4390,104.6000,94.3000,0.0000,104.2000,87.9000,0.0000,104.1000,87.5000,0.0000,104.7000,97.2000,0.0000,106.1000,100.0000,0.0000,106.7000,100.0000,0.0000,107.3000,100.0000,0.0000,107.4000,100.0000,0.0000,106.6000,87.4000,0.0000,106.2000,81.2000,0.0000,106.3000,82.3000,0.0000,106.4000,83.7000,0.0000,105.6000,71.7000,0.0000,0.0000,0.0000,0.0000
4,1968-01-31,909.3870,105.0000,62.7000,12.3000,104.9000,60.5000,11.3000,104.2000,49.2000,10.1000,104.2000,49.3000,9.2000,104.3000,51.9000,8.7000,103.6000,40.9000,7.9000,104.1000,48.7000,7.7000,104.1000,48.5000,7.5000,104.7000,57.4000,7.5000,105.9000,76.7000,8.5000,107.2000,96.9000,9.8000,108.5000,100.0000,11.7000,108.3000,97.3000,13.1000,13.1000,13.1000,13.1000


In [8]:
# difference transformation
# df = df.diff()
# df = df.drop(index='1967-02-09')

In [9]:
# Histograms of features
features = []
for i in range(14):
    features.extend(['BCI_{i}', 'BCIp_{i}', 'BCIg{i}'])

def plot():
    for feature in features:
        df.hist(column = feature, bins = 50)
        plt.xlabel(feature,fontsize=15)
        plt.ylabel("Frequency",fontsize=15)
        plt.show()
        df.plot(y=feature)

# plot()

In [10]:
df.to_csv("../merged_data/BCI_GDP.csv")